<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Inteligencia Artificial</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Incertidumbre</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---


# Objetivo <a class="anchor" id="inicio"></a>

En este notebook estudiaremos una herramienta muy útil para representar situaciones de incertidumbre como las que aparecen, por ejemplo, en el mundo del Wumpus, aunque el rango de aplicaciones es muy amplio. Esta herramienta se llama las **redes bayesianas** y está basada, como su nombre lo indica, en la regla de Bayes. También veremos cómo tomar decisiones racionales, entendidas estas como aquellas que maximizan la utilidad esperada del agente. Veremos que podemos echar mano de las redes bayesianas, pero necesitamos también el concepto de las funciones de utilidad.

Adaptado de Russell & Norvig (2016), caps. 13, 14 y 16.


[Ir a ejercicio 1](#ej1)

## Dependencias

Al iniciar el notebook o reiniciar el kerner se pueden cargar todas las dependencias de este notebook corriendo las siguientes celdas. Este también es el lugar para instalar las dependencias que podrían hacer falta.

**De Python:**

In [ ]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
import warnings
warnings.filterwarnings('ignore')
gum.config['notebook','potential_visible_digits']=2

Observe que pyAgrum requiere tener instalado el programa [graphviz](https://graphviz.org/). Si no está instalado en su equipo, en Ubuntu puede correr la siguiente celda:

In [ ]:
#!apt install graphviz

Para instalación en otras versiones de Linux u otros sistemas operativos puede consultar la [documentación de graphviz](https://graphviz.org/download/).

**Del notebook:**

In [ ]:
from ambientes import Wumpus
from agentes import HeroeUE
import utils

# Secciones

* [Implementación de la estructura de una red bayesiana.](#estructura)
* [Inferencia probabilística.](#inferencia)
* [Ejemplos.](#ejemplos)
* [Utilidad esperada y toma de decisiones.](#MEU)
* [Implementación de redes de decisión](#implementacion)
* [Escenarios más complejos.](#dependencia)
* [Agente basado en utilidad.](#utility-based)


# Estructura de una red bayesiana  <a class="anchor" id="estructura">

([Volver al inicio](#inicio))
    
Una **red bayesiana** está basada en un grafo dirigido acíclico, en donde cada vértice está etiquetado con una tabla de probabilidad condicional en función de su ascendencia. Para la implementación aquí realizada, usaremos la librería [pyAgrum](https://pyagrum.readthedocs.io/en/0.22.2/). 
    
Usaremos como ejemplo el razonamiento sobre los pozos en el mundo del Wumpus discutido en las diapositivas. Lo primero que debemos hacer es crear un objeto de clase `BayesNet`. En este ejemplo inicializaremos este objeto mediante el método `fastBN`, el cual permite incluir el grafo subyacente, toda vez que este es bastante sencillo:

In [ ]:
bn=gum.fastBN('Pozo(2,0)->Brisa(1,0)<-Pozo(1,1)')
gnb.showBN(bn,size="6")

Debemos ahora incluir las probabilidades de los pozos y la probabilidad condicional de la brisa. Esto se realiza de la siguiente manera:

In [ ]:
# Usando fillWith
bn.cpt('Pozo(2,0)').fillWith([0.8,0.2])
bn.cpt('Pozo(1,1)').fillWith([0.8,0.2])
# Usando un diccionario
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 1, 'Pozo(1,1)': 1}] = [0, 1]
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 1, 'Pozo(1,1)': 0}] = [0, 1]
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 0, 'Pozo(1,1)': 1}] = [0, 1]
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 0, 'Pozo(1,1)': 0}] = [1, 0]

Y se visualiza bonito de la siguiente manera:

In [ ]:
gnb.sideBySide(bn.cpt('Pozo(2,0)'),
               gnb.getBN(bn),
               bn.cpt('Pozo(1,1)'))
gnb.sideBySide(bn.cpt('Brisa(1,0)'),captions=['$P(Brisa{(1,0)}|Pozo{(2,0)},Pozo{(1,1)})$'])

# Inferencia probabilística <a class="anchor" id="inferencia">

([Volver al inicio](#inicio))
    
Con esta red bayesiana podemos resolver consultas sobre cualquier cálculo de probabilidades en este escenario. Las consultas consisten en saber cuál es la probabilidad de que una variable (o variables) tomen un valor (o valores) determinados.

Para realizar una consulta, es necesario crear un objeto de consulta mediante el método `LazyPropagation` aplicado sobre nuestra red bayesiana `bn`:

In [ ]:
ie = gum.LazyPropagation(bn)

Consideraremos dos tipos de consultas. El primero es cuando no hay evidencia disponible. En este caso, la respuesta a la consulta es simplemente la **probabilidad marginal** de que las variables tomen los valores respectivos. 

Por ejemplo, podemos consultar la probabilidad de que haya un pozo en $(2,0)$ de la siguiente manera:

In [ ]:
ie.makeInference()
ie.posterior('Pozo(2,0)')

El segundo tipo es cuando hay evidencia disponible $E$ (que viene dada en forma de un diccionario `{variable:valor}`). Aquí, la respuesta a la consulta es la **probabilidad condicional** de que las variables tomen los valores respectivos dada la evidencia. La evidencia se guardará mediante el método `setEvidence`. Por ejemplo, suponga que la evidencia es que hay brisa en $(1,0)$. La evidencia en este caso se guarda de la siguiente manera:

In [ ]:
ie.setEvidence({'Brisa(1,0)':1})

Y volvemos a hacer la consulta:

In [ ]:
ie.makeInference()
ie.posterior('Pozo(2,0)')

La librería pyAgrum es muy flexible y viene con métodos muy interesantes para visualización. Por ejemplo, podemos visualizar lado a lado las probabilidades a priori y a posteriori de $Pozo{(2,0)}$:

In [ ]:
print("Probabilidad a priori Pozo(2,0):")
gnb.showInference(bn,evs={})
print("Probabilidad a posteriori dado Brisa(1,0):")
gnb.showInference(bn,evs={'Brisa(1,0)':1})

# Ejemplos <a class="anchor" id="ejemplos">
    
([Volver al inicio](#inicio))

### Regar el jardín

Considere la siguiente situación. Para tener un bonito jardín, es necesario que este esté húmedo la mayor parte del tiempo. La rutina diaria de María es chequear el clima. Si está nublado, usualmente no prende el roceador automático del jardín. Si no está nublado, a veces sí y a veces no lo prende. Si el roceador está funcionando o si llueve, el jardín estará mojado. Estas relaciones se ven reflejadas en la siguiente red bayesiana:

<img src="./imagenes/jardin.png" width="650">

Para hacer inferencias probabilisticas sobre esta situación, creamos el objeto `BayesNet`. Vamos a inicializar el grafo dirigido acíclico de manera rápida mediante la siguiente celda:

In [ ]:
bn=gum.fastBN("Nublado->Roceador->JardinMojado<-Lluvia<-Nublado")
gnb.showBN(bn,size="6")

<a class="anchor" id="ej1"></a>**Ejercicio 1:** 

([Próximo ejercicio](#ej2))

Asigne las probabilidades de cada uno de los nodos y dibuje la red correspondiente:

In [ ]:
# Creamos la red
bn=gum.fastBN("Nublado->Roceador->JardinMojado<-Lluvia<-Nublado")

# AQUI PROBABILIDADES PARA LA VARIABLE Nublado

# AQUI PROBABILIDADES PARA LA VARIABLE Roceador

# AQUI PROBABILIDADES PARA LA VARIABLE Lluvia

# AQUI PROBABILIDADES PARA LA VARIABLE JardinMojado

# Dibuja la red
gnb.sideBySide(bn.cpt('Nublado'))
gnb.sideBySide(
  gnb.getSideBySide(bn.cpt('Roceador')),
  gnb.getBN(bn,size="3!"),
  gnb.getSideBySide(bn.cpt('Lluvia')))
gnb.sideBySide(bn.cpt('JardinMojado'))

Al correr la celda se debe obtener lo siguiente:

<img src="./imagenes/jardin1.png" width="600">

---

<a class="anchor" id="ej2"></a>**Ejercicio 2:** 

([Anterior ejercicio](#ej1)) ([Próximo ejercicio](#ej3))

Utilice un objeto `LazyPropagation` y los métodos `makeInference`, `posterior` y `setEvidence` para responder las siguientes preguntas sobre la situación del jardín:

* ¿Cuál es la probabilidad de que el jardín esté mojado? $P(JardinMojado)$
* Dado que está nublado, ¿cuál es la probabilidad de que el jardín esté mojado? $P(JardinMojado|Nublado)$
* Dado que el jardín está mojado, cuál es la probabilidad de que el roceador esté encendido? $P(Roceador|JardinMojado)$

La figura a obtener es la siguiente:

<img src="./imagenes/jardin2.png" width="600">

---

### Alarma antirobos

Usted ha instalado un sistema de alarma para su casa, el cual es muy confiable para detectar robos, pero en ciertas ocasiones también responde a terremotos leves. Usted tiene dos vecinos, Juan y María, que han prometido llamarlo a su trabajo cuando escuchen la alarma. Juan casi siempre llama cuando escucha la alarma, pero a veces confunde el sonido del teléfono con la alarma y hará la llamada. Por otro lado, María escucha música a todo volumen y por eso algunas veces no escucha la alarma. Las probabilidades y dependencias de esta situación se pueden representar mediante la siguiente red bayesiana:

<img src="./imagenes/alarma.png" width="600">

<a class="anchor" id="ej3"></a>**Ejercicio 3:** 

([Anterior ejercicio](#ej2)) ([Próximo ejercicio](#ej4))

* Implemente la red bayesiana correspondiente a esta situación y dibuje la red. 
* Utilice la red bayesiana para determinar las siguientes probabilidades:
    * Probabilidad de que suene la alarma.
    * Probabilidad de que haya un robo dado que Juan y María llaman.
    
Use la siguiente instrucción para visualizar tres dígitos de las probabilidades:

In [ ]:
gum.config['notebook','potential_visible_digits']=3

---

### Más Wumpus
    
Vamos a resolver ahora el ejemplo del mundo del Wumpus con el que terminamos las diapositivas de clase:
    
<img src="./imagenes/pozos.png" width="450">
    
Para poder hacer este cálculo, debemos crear la red bayesiana respectiva, representada en el siguiente diagrama:
    
<img src="./imagenes/rb1.png" width="600">   
    
Para ilustrar otros métodos de creación de redes bayesianas en PyAgrum, vamos a crear una red con los nodos, aristas y probabilidades requeridas.

In [ ]:
# Creamos la red
bn=gum.BayesNet()

# Creamos las variables
variables = ['Pozo(2,0)', 'Pozo(1,1)', 'Pozo(0,2)', 'Brisa(1,0)', 'Brisa(0,1)']
for var in variables:
    bn.add(var, 2) # <= aquí el 2 es el número de posibles valores

gnb.showBN(bn,size="6")

Añadimos las aristas:

In [ ]:
bn.addArc('Pozo(0,2)','Brisa(0,1)')
bn.addArc('Pozo(1,1)','Brisa(0,1)')
bn.addArc('Pozo(1,1)','Brisa(1,0)')
bn.addArc('Pozo(2,0)','Brisa(1,0)')
gnb.showBN(bn,size="6")

Y añadimos las probabilidades:

In [ ]:
# Usando fillWith
bn.cpt('Pozo(2,0)').fillWith([0.8,0.2])
bn.cpt('Pozo(1,1)').fillWith([0.8,0.2])
bn.cpt('Pozo(0,2)').fillWith([0.8,0.2])
# Usando un diccionario
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 1, 'Pozo(1,1)': 1}] = [0, 1]
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 1, 'Pozo(1,1)': 0}] = [0, 1]
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 0, 'Pozo(1,1)': 1}] = [0, 1]
bn.cpt('Brisa(1,0)')[{'Pozo(2,0)': 0, 'Pozo(1,1)': 0}] = [1, 0]
bn.cpt('Brisa(0,1)')[{'Pozo(1,1)': 1, 'Pozo(0,2)': 1}] = [0, 1]
bn.cpt('Brisa(0,1)')[{'Pozo(1,1)': 1, 'Pozo(0,2)': 0}] = [0, 1]
bn.cpt('Brisa(0,1)')[{'Pozo(1,1)': 0, 'Pozo(0,2)': 1}] = [0, 1]
bn.cpt('Brisa(0,1)')[{'Pozo(1,1)': 0, 'Pozo(0,2)': 0}] = [1, 0]

In [ ]:
gnb.sideBySide(bn.cpt('Pozo(0,2)'), bn.cpt('Pozo(1,1)'), bn.cpt('Pozo(2,0)')),
gnb.sideBySide(gnb.getBN(bn)),
gnb.sideBySide(bn.cpt('Brisa(0,1)'),
               gnb.sideBySide(captions=['']),
               bn.cpt('Brisa(1,0)'),
               captions=['$P(Brisa_{(0,1)}|Pozo_{(0,2)},Pozo_{(1,1)})$', '', '$P(Brisa_{(1,0)}|Pozo_{(2,0)},Pozo_{(1,1)})$'])


Ahora ya podemos encontrar la probabilidad de $Pozo(1,1)$ dado que $Brisa(1,0)$ y $Brisa(0,1)$:

In [ ]:
ie=gum.LazyPropagation(bn)
ie.setEvidence({'Brisa(1,0)':1, 'Brisa(0,1)':1})
ie.makeInference()
ie.posterior('Pozo(1,1)')

In [ ]:
print("Probabilidad a priori:")
gnb.showInference(bn,evs={})
print("Probabilidad a posteriori incluyendo Brisa(1,0):")
gnb.showInference(bn,evs={'Brisa(1,0)':1})
print("Nueva probabilidad a posteriori incluyendo Brisa(1,0)' y 'Brisa(0,1)")
gnb.showInference(bn,evs={'Brisa(1,0)':1, 'Brisa(0,1)':1})

Aquí se pueden observar cosas muy interesantes. Por ejemplo, note que la probabilidad de $Pozo{(2,0)}$ comienza en su valor original de 0.2. Al incluir la evidencia $Brisa{(1,0)}$ la probabilidad de $Pozo{(2,0)}$ aumenta a 0.56, pero luego disminuye a 0,31 cuando incluimos la evidencia de $Brisa{(0,1)}$.

---

<a class="anchor" id="ej4"></a>**Ejercicio 4:** 

([Anterior ejercicio](#ej3)) ([Próximo ejercicio](#ej5))

Cree una red bayesiana para representar la relación entre el Wumpus y el hedor. Observe que esta relación se diferencia de la que existe entre la brisa y los posos en que sólo hay un Wumpus. Para hacer la implementación, considere:

* Crear una variable 'Wumpus' con 16 posibles valores.
* Crear una variable 'Hedor(x, y)' para cada una de las 16 casillas.
* Crear los arcos respectivos.
* Crear la tabla de probabilidad condicional de cada variable 'Hedor(x, y)', que debe valer 1 si el valor de la variable 'Wumpus' (que es una casilla) es una casilla adyacente a $(x, y)$ y es 0 en caso contrario.

Al correr las siguientes celdas se debe obtener:

<img src="./imagenes/wumpus_cell.png" width="400"/>

y

<img src="./imagenes/hedor_cell.png" width="90"/>



In [ ]:
bn.cpt('Wumpus')

In [ ]:
bn.cpt('Hedor(1, 1)')

---

<a class="anchor" id="ej5"></a>**Ejercicio 5:** 

([Anterior ejercicio](#ej4)) ([Próximo ejercicio](#ej6))

Compruebe que la inferencia a partir de la evidencia de que hay hedor en (1, 1) es que las casillas 1, 4, 6 y 9 tienen probabilidad 0.25 de que en alguna de ellas esté el Wumpus. 

---

# Utilidad esperada y toma de decisiones   <a class="anchor" id="MEU">

([Volver al inicio](#inicio))
    
Hemos visto que uno de los objetivos de la Inteligencia Artificial es producir agentes cuyo comportamiento sea inteligente. Este se ha entendido tradicionalmente como la realización de acciones racionales, las cuales  buscan obtener fines de la manera más eficiente posible. Imagine que un agente debe tomar una decisión de entre un conjunto de acciones $\{a_1,\ldots, a_0\}$. Cada acción $a_i$ produce un estado $S_i$. Si el agente fuera indiferente frente a los estados $S_i$s, entonces cualquier acción estaría bien para él y el problema de decisión sería trivial. No obstante, un agente racional tiene un objetivo que quiere cumplir. Esto divide los estados $S_i$s en aquellos que cumplen el objetivo y aquellos que no. Es más, muchas veces es posible asignar una preferencia entre dos estados, $S_i$ y $S_j$, de tal manera que el agente prefiere a $S_i$ sobre $S_j$ si el primero lo acerca más a su objetivo que el segundo. Así pues, un agente racional, frente a su problema de decisión, realizará la acción $a$ que con mayor probabilidad lo lleve al estado que esté más cerca del objetivo.
    
Es posible formalizar esta idea usando tres conceptos. El primero es el de una *lotería*, el cual no es otra cosa que un conjunto de estados, cada uno con una probabilidad asignada: $\{S_1,p_1; S_2,p_2; \ldots; S_n,p_n\}$. El segundo es el de una *función de utilidad* $U$, la cual le asigna un valor numérico a cada estado $S_i$ de tal manera que $U(S_i)>U(S_j)$ sii el agente prefiere el estado $S_i$ sobre el estado $S_j$. Estos dos conceptos nos permiten definir la *Utilidad Esperada* de la siguiente manera:
    
$$U\!E=\sum_{i=1}^n U(S_i)p_i$$
    
El tercer concepto es que el conjunto de estados $\{S_1,p_1; S_2,p_2; \ldots; S_n,p_n\}$ depende de una acción $a$. Es decir, se representan situaciones en las cuales el resultado de una acción $a$ es incierta. La incertidumbre resulta porque la acción no determina completamente el resultado cuando, por ejemplo, este depende de otros agentes, o de factores que no son completamente conocidos y/o controlables por el agente. Esto es, una acción $a$ da como resultado un estado $S_1$ con probabilidad $P(S_1|a)$, o un estado $S_2$ con probabilidad $P(S_2|a)$, $\ldots$, o un estado $S_n$ con probabilidad $P(S_n|a)$. Así pues, tenemos el concepto de *Utilidad Esperada* de una acción $a$:
    
$$U\!E(a)=\sum_{i=1}^n U(S_i)P(S_i|a)\qquad\qquad (1)$$

Un agente racional, frente a un problema de decisión, decidirá ejecutar la acción que le permita **maximizar su utilidad esperada**:
    
$$\mbox{Acción}=\mbox{argmax}_a\, U\!E(a)=\mbox{argmax}_a\sum_{i=1}^n U(S_i)P(S_i|a)\qquad\qquad (2)$$

## Ejemplo con el mundo del Wumpus

Para ilustrar esta propuesta de cómo un agente toma una decisión racional, volvamos a nuestro bien conocido mundo del Wumpus. Suponga que el agente, después de partir de la celda $(0,0)$, se encuentra en la casilla $(1,0)$. Analicemos dos situaciones posibles, como lo muestran las siguientes figuras:

<table>
  <tr>
    <td><img src="./imagenes/ParaRedesDecisionB.png" width="300"></td>
    <td><img src="./imagenes/ParaRedesDecisionA.png" width="300"></td>
  </tr>
</table>

Las dos situaciones corresponden a si el agente siente una brisa (izquierda) o no (derecha). ¿Cuál es la acción racional en cada caso? Observe que el agente tiene tres acciones posibles:

1. Regresarse a $(0,0)$
2. Seguir a $(2,0)$
3. Subir a $(1,1)$.


Hagamos el ejercicio de asignar utilidades a los estados resultantes de estas acciones. Estos estados son, claramente, la casilla a la que llega el agente después de realizar la acción. Vamos a proceder a determinar la utilidad con base en la consideración de si en la casilla correspondiente hay o no un pozo y de si está o no el oro. Por ejemplo:

$$
U(\mbox{casilla}(x,y)) = \begin{cases}
1, & \mbox{si }oro(x,y) \wedge \neg pozo(x,y)\\
0, & \mbox{si }\neg oro(x,y) \wedge \neg pozo(x,y)\\
-1, & \mbox{ si }pozo(x,y)
\end{cases}
$$

Dado que el agente desconoce la localización de los pozos y del oro, no es posible deducir cuál es la utilidad de una casilla. No obstante, como vimos en el notebook pasado, sí podemos cuantificar la incertidumbre y revisar nuestras predicciones con base en la información que vamos incorporando. Podemos entonces ponderar la utilidad de cada estado posible con su probabilidad de ocurrencia, mediante la fórmula de la utilidad esperada (1). En este caso tendríamos:

\begin{align*}
U\!E(\mbox{ir a casilla }(x,y)|e)&=U(oro(x,y), \neg pozo(x,y))P(oro(x,y), \neg pozo(x,y)|e)\\
& \quad + U(\neg oro(x,y), \neg pozo(x,y))P(\neg oro(x,y), \neg pozo(x,y)|e)\\
&=U(oro(x,y), \neg pozo(x,y))P(oro(x,y))P(\neg pozo(x,y)|e)\\
& \quad + U(\neg oro(x,y), \neg pozo(x,y))P(\neg oro(x,y))P(\neg pozo(x,y)|e)\\
& \quad + U(pozo(x,y))P(pozo(x,y)|e)\\
&= 1\times P(oro(x,y))P(\neg pozo(x,y)|e)\\
& \quad + 0 \times P(\neg oro(x,y))P(\neg pozo(x,y)|e)\\
& \quad - 1 \times P(pozo(x,y)|e)
\end{align*}

La acción racional queda determinada por la fórmula (2) de maximización de la utilidad esperada, la cual requiere que realicemos el cálculo anterior para todas las acciones posibles, de tal manera que escogeremos la acción que tenga asociada la mayor utilidad esperada. 

Ahora bien, observe que llevar a cabo estos cálculos puede hacerse con o sin evidencia. Observe también que la evidencia disponible es lo que separa la situación de la figura de arriba a la izquierda (en donde la evidencia es que hay brisa en $(1,0)$) de la situación de la derecha (en donde no hay brisa). Podríamos tratar de hacer los cálculos a mano, pero es más divertido programar el computador para que los haga por nosotros.

Vamos a ver a continuación cómo usar el objeto `InfluenceDiagram` de la librería `pyagrum` para implementar el cálculo de la toma racional de decisiones. Comenzaremos con una situación en que no consideramos el oro, que es más sencilla, para luego incluir esta complicación en nuestros cálculos.

# Implementación de redes de decisión <a class="anchor" id="implementacion">
    
([Volver al inicio](#inicio))

Una red de decisión, también conocida como un diagrama de influencia, es una red que incorpora una red bayesiana, un nodo de acción y otro de utilidad (es posible tener más de un nodo de acción y más de un nodo de utilidad, pero en nuestros ejemplos solo tendremos uno de cada uno). Mediante ella es posible calcular las utilidades esperadas de las acciones de acuerdo a las probabilidades establecidas en la red bayesiana y la evidencia disponible.
    
En la red de decisión hay tres tipos de nodos:
    
* Nodos de probabilidad: son los nodos de la red bayesiana.
* Nodo de decisión: es el nodo que representa las acciones posibles.
* Nodo de utilidad: está conectado a los demás nodos de tal manera que representa la utilidad esperada de cada acción del nodo de decisión, de acuerdo a las probabilidades de los nodos de probabilidad.
 
Para comprender mejor estos conceptos, creemos una red de decisión simple para el ejemplo del mundo del Wumpus que discutimos anteriormente. 

### Ejemplo sin considerar el oro <a class="anchor" id="sin_oro">
    
([Siguiente ejemplo](#solo_oro))



Vamos a comenzar con un caso sencillo, en el cual no tomamos en cuenta si hay o no oro en una casilla. De esta manera, nuestra función de utilidad es la siguiente:
    
$$
U(\mbox{casilla}(x,y)) = \begin{cases}
1, & \mbox{si }\neg pozo(x,y)\\
-1, & \mbox{ si }pozo(x,y)
\end{cases}
$$
    
Creamos una instancia de la clase `InfluenceDiagram`:   

In [ ]:
model = gum.InfluenceDiagram()

Ahora creamos los nodos de probabilidad y sus conexiones. Observe que esto es muy similar a la manera como creamos una red bayesiana en el notebook pasado:

**Red bayesiana con los nodos de probabilidad:**

In [ ]:
# Creando los nodos
pozo0 = gum.LabelizedVariable('Pozo(0,0)','Pozo(0,0)',2)
model.addChanceNode(pozo0)
pozo1 = gum.LabelizedVariable('Pozo(1,1)','Pozo(1,1)',2)
model.addChanceNode(pozo1)
pozo2 = gum.LabelizedVariable('Pozo(2,0)','Pozo(2,0)',2)
model.addChanceNode(pozo2)
brisa = gum.LabelizedVariable('Brisa(1,0)','Brisa(1,0)',2)
model.addChanceNode(brisa)

# Creando las aristas de la red bayesiana
model.addArc('Pozo(0,0)', 'Brisa(1,0)')
model.addArc('Pozo(1,1)', 'Brisa(1,0)')
model.addArc('Pozo(2,0)', 'Brisa(1,0)')

# Creando las probabilidades
model.cpt('Pozo(0,0)')[:]=[0.8,0.2]
model.cpt('Pozo(1,1)')[:]=[0.8,0.2]
model.cpt('Pozo(2,0)')[:]=[0.8,0.2]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 1, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 1, 'Pozo(2,0)': 0}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 0, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 0, 'Pozo(2,0)': 0}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 1, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 1, 'Pozo(2,0)': 0}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 0, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 0, 'Pozo(2,0)': 0}] = [1, 0]

In [ ]:
gnb.showInfluenceDiagram(model,size="6")

**Nodo de decisión:**

Luego creamos un nodo de decisión. Este nodo es la variable que representa cuál es la acción a tomar. En nuestro ejemplo, definir una acción es equivalente a considerar a cuál casilla se va a mover el agente.

In [ ]:
casilla = gum.LabelizedVariable('Casilla','Casilla a moverse',3)
casilla.changeLabel(0,'(0,0)')
casilla.changeLabel(1,'(2,0)')
casilla.changeLabel(2,'(1,1)')
model.addDecisionNode(casilla)

**Nodo de utilidad:**

Y ahora creamos el nodo de utilidad. Las conexiones a este nodo deben ser todas las variables sobre las cuales depende la utilidad del estado obtenido por la acción realizada. En nuestro caso, consideramos los pozos y la casilla a la que se mueve el agente:

In [ ]:
ut_casilla = gum.LabelizedVariable('UtilityOfCasilla','Valor casilla',1)
model.addUtilityNode(ut_casilla)

model.addArc('Pozo(0,0)', 'UtilityOfCasilla')
model.addArc('Pozo(1,1)', 'UtilityOfCasilla')
model.addArc('Pozo(2,0)', 'UtilityOfCasilla')
model.addArc('Casilla', 'UtilityOfCasilla')

gnb.showInfluenceDiagram(model,size="6")

Nos falta incluir el cálculo de la utilidad de cada acción (que en nuestro caso es una casilla). Definamos primero la función de utilidad de una casilla:

In [ ]:
def utilidad(d):
    '''
    Función que determina la utilidad de una casilla.
    Input: d, que es un diccionario de la forma:
        {
            'Pozo(0,0)':valor1,
            'Pozo(1,1)':valor2,
            'Pozo(2,0)':valor3,
            'Casilla':'(x,y)'
        }
    '''
    
    C = d['Casilla']
    if (d['Pozo' + C] == 0):
        return 1
    else:
        return -1

Comprobamos el funcionamiento de la función con un ejemplo:

In [ ]:
d = {'Pozo(0,0)':1,'Pozo(1,1)':0,'Pozo(2,0)':1,'Casilla':'(0,0)'}
utilidad(d)

Ahora debemos asignar la utilidad al nodo `UtilityOfCasilla` de acuerdo a todas las posibles combinaciones de valores para las variables que conectan a ella. Pero para hacer esto de manera escalable, crearemos automáticamente todos los diccionarios con todas las combinaciones de valores posibles.

In [ ]:
variables = ['Pozo(0,0)','Pozo(1,1)','Pozo(2,0)','Casilla']
opciones = product([0,1], [0,1], [0,1], ['(0,0)','(1,1)','(2,0)'])
dicts_variables = [{variables[i]:op[i] for i in range(len(variables))} for op in opciones]
dicts_variables

Asignamos la utilidad del nodo `UtilityOfCasilla`:

In [ ]:
for d in dicts_variables:
    model.utility('UtilityOfCasilla')[d]=utilidad(d)

Y finalmente comprobamos que el cálculo de la utilidad sea el esperado:

In [ ]:
ie = gum.InfluenceDiagramInference(model)
ie.makeInference()
ie.posteriorUtility("UtilityOfCasilla")

Ya podemos visualizar la red de decisión:

In [ ]:
print("Situación I: Análisis del escenario sin información:")
gnb.showInference(model,evs={},size="6")

Observe que el nodo de decisión `Casilla` nos muestra la utilidad esperada de cada opción. En este caso, todas tienen un valor de 0.6. La mejor opción también está señalada (que en este caso es la primera que aparece).

Ahora bien, debemos incluir la evidencia que tengamos a nuestra disposición. Ya sabemos que no hay pozo en la casilla $(0,0)$ porque las reglas del mundo lo prohiben. También, dependiendo del escenario que consideremos (con brisa o sin brisa), podemos ver qué acción tiene mayor utilidad esperada. Tómese unos minutos para entender el análisis siguiente de las dos situaciones visualizadas en la red de decisión:

In [ ]:
print("Situación II: Análisis del escenario con brisa:")
gnb.showInference(model,evs={'Pozo(0,0)':0,'Brisa(1,0)':1},size="6")
print("Situación III: Análisis del escenario sin brisa:")
gnb.showInference(model,evs={'Pozo(0,0)':0,'Brisa(1,0)':0},size="6")

Tanto en la situación I como en la III, tenemos varias acciones con la misma utilidad esperada: un empate entre máximos. El método `optimalDecision` que viene con la librería siempre escogerá la primera acción en este empate:

In [ ]:
ie = gum.InfluenceDiagramInference(model)
ie.setEvidence({'Pozo(0,0)':0,'Brisa(1,0)':0})
ie.makeInference()
print("Utilidades esperadas de las acciones:")
print(ie.posteriorUtility("Casilla"))
print("Decisión de la librería:")
print(ie.optimalDecision("Casilla"))

Podemos crear una sencilla función auxiliar para escoger una acción aleatoria entre aquellas que tienen máxima utilidad esperada:

In [ ]:
def maximo_aleatorio(valores):
    indices = [i for i, x in enumerate(valores) if x == max(valores)]
    return np.random.choice(indices)

In [ ]:
casillas = ['(0,0)', '(1,1)', '(2,0)']
valores = ie.posteriorUtility("Casilla").tolist()
accion = maximo_aleatorio(valores)
print("Primera iteración:", casillas[accion])
accion = maximo_aleatorio(valores)
print("Segunda iteración:", casillas[accion])
accion = maximo_aleatorio(valores)
print("Tercera iteración:", casillas[accion])

<a class="anchor" id="ej6"></a>**Ejercicio 6:** 

([Anterior ejercicio](#ej5)) ([Próximo ejercicio](#ej7))

Haga un análisis de las utilidades esperadas y de la acción racional a tomar cuando el agente sabe que no hay pozo en $(0,0)$ pero sí en $(2,0)$.

---

### Ejemplo solo incluyendo el oro <a class="anchor" id="solo_oro">
    
([Anterior ejemplo](#sin_oro)  -- [Siguiente ejemplo](#combinado))

Antes de incluir tanto pozos como oro en una red de decisión, consideremos primero un ejemplo en donde sólo incluimos el oro. Junto con el ejemplo anterior, ya sabremos cómo influyen estos dos factores en la utilidad y nos será más claro analizar el caso combinado.
    
Comenzamos creando la parte correspondiente a la red bayesiana, que en este caso relaciona una variable `oro`, que representa la localización del oro en la rejilla, con las variables binarias que representan si en una casilla hay o no brillo:

In [ ]:
model = gum.InfluenceDiagram()

# Creamos los nodos de probabilidad
oro0 = gum.LabelizedVariable('Brillo(0,0)','Brillo(0,0)',2)
model.addChanceNode(oro0)
oro1 = gum.LabelizedVariable('Brillo(1,1)','Brillo(1,1)',2)
model.addChanceNode(oro1)
oro2 = gum.LabelizedVariable('Brillo(2,0)','Brillo(2,0)',2)
model.addChanceNode(oro2)
oro = gum.LabelizedVariable('Oro','Oro',3)
oro.changeLabel(0,'(0,0)')
oro.changeLabel(1,'(1,1)')
oro.changeLabel(2,'(2,0)')
model.addChanceNode(oro)

# Creamos las aristas de la red bayesiana
model.addArc('Oro', 'Brillo(0,0)')
model.addArc('Oro', 'Brillo(1,1)')
model.addArc('Oro', 'Brillo(2,0)')

# Creamos las probabilidades
model.cpt('Oro').fillWith([1/3,1/3,1/3])
model.cpt('Brillo(0,0)')[{'Oro': 0}] = [0,1]
model.cpt('Brillo(0,0)')[{'Oro': 1}] = [1,0]
model.cpt('Brillo(0,0)')[{'Oro': 2}] = [1,0]
model.cpt('Brillo(1,1)')[{'Oro': 0}] = [1,0]
model.cpt('Brillo(1,1)')[{'Oro': 1}] = [0,1]
model.cpt('Brillo(1,1)')[{'Oro': 2}] = [1,0]
model.cpt('Brillo(2,0)')[{'Oro': 0}] = [1,0]
model.cpt('Brillo(2,0)')[{'Oro': 1}] = [1,0]
model.cpt('Brillo(2,0)')[{'Oro': 2}] = [0,1]

gnb.showInfluenceDiagram(model,size="6")

Incluimos ahora el nodo de decisión, el de utilidad, y las relaciones de utilidades. La utilidad, cuando sólo consideramos el oro, estará dada por la siguiente fórmula:

$$
U(\mbox{casilla}(x,y)) = \begin{cases}
1, & \mbox{si }Oro(x,y)\\
0, & \mbox{ si }\neg Oro(x,y)
\end{cases}
$$

In [ ]:
# Creamos el nodo de decisión
casilla = gum.LabelizedVariable('Casilla','Casilla a moverse',3)
casilla.changeLabel(0,'(0,0)')
casilla.changeLabel(1,'(2,0)')
casilla.changeLabel(2,'(1,1)')
model.addDecisionNode(casilla)

# Creamos el nodo de utilidad
ut_casilla = gum.LabelizedVariable('UtilityOfCasilla','Valor casilla',1)
model.addUtilityNode(ut_casilla)

# Creamos las aristas de la utilidad
model.addArc('Oro', 'UtilityOfCasilla')
model.addArc('Casilla', 'UtilityOfCasilla')

# Asignamos las utilidades
variables = ['Oro','Casilla']
opciones = product(['(0,0)','(1,1)','(2,0)'], ['(0,0)','(1,1)','(2,0)'])
dicts_variables = [{variables[i]:op[i] for i in range(len(variables))} for op in opciones]

def utilidad(d):
    C = d['Casilla']
    if d['Oro'] == C:
        return 1
    else:
        return 0

for d in dicts_variables:
    model.utility('UtilityOfCasilla')[d]=utilidad(d)

gnb.showInfluenceDiagram(model,size="6")

Confirmamos que el cálculo de la utilidad esté correcto:

In [ ]:
ie = gum.InfluenceDiagramInference(model)
ie.makeInference()
ie.posteriorUtility("UtilityOfCasilla")

Consideremos ahora el escenario sin evidencia:

In [ ]:
print("Situación I: Análisis del escenario sin evidencia:")
gnb.showInference(model,evs={},size="6")

En la situación I, en la cual no tenemos evidencia, cualquiera de las tres casillas representa la misma utilidad esperada. Esto coincide con nuestra intuición, toda vez que el oro puede estar en cualquiera de ellas. 

<a class="anchor" id="ej7"></a>**Ejercicio 7:** 

([Anterior ejercicio](#ej6)) ([Próximo ejercicio](#ej8))

Analice el escenario en el que no hay brillo en $(0,0)$. ¿Qué casilla tiene mayor utilidad esperada? ¿Es esto intuitivamente cierto?

---

# Escenarios más complejos  <a class="anchor" id="dependencia">
    
([Volver al inicio](#inicio))
    
Ya estamos listos para considerar un escenario que incluye tanto los pozos y la briza como el oro y el brillo. Pero haremos esto en dos pasos: primero ambas variables en un escenario pequeño, luego el escenario completo que se requiere para implementar un agente basado en utilidad. 

    
### Oro y pozos <a class="anchor" id="combinado">
    
El siguiente es el código que genera la red de decisión. Observe la creación de la red bayesiana, del nodo de decisión con las acciones del agente, y el nodo de utilidad con la utilidad que definimos al comienzo del notebook.
    
([Anterior ejemplo](#solo_oro)  -- [Siguiente ejemplo](#combinado_todos))


In [ ]:
model = gum.InfluenceDiagram()

# Creamos los nodos de probabilidad
pozo0 = gum.LabelizedVariable('Pozo(0,0)','Pozo(0,0)',2)
model.addChanceNode(pozo0)
pozo1 = gum.LabelizedVariable('Pozo(1,1)','Pozo(1,1)',2)
model.addChanceNode(pozo1)
pozo2 = gum.LabelizedVariable('Pozo(2,0)','Pozo(2,0)',2)
model.addChanceNode(pozo2)
brisa = gum.LabelizedVariable('Brisa(1,0)','Brisa(1,0)',2)
model.addChanceNode(brisa)
oro0 = gum.LabelizedVariable('Brillo(0,0)','Brillo(0,0)',2)
model.addChanceNode(oro0)
oro1 = gum.LabelizedVariable('Brillo(1,1)','Brillo(1,1)',2)
model.addChanceNode(oro1)
oro2 = gum.LabelizedVariable('Brillo(2,0)','Brillo(2,0)',2)
model.addChanceNode(oro2)
oro = gum.LabelizedVariable('Oro','Oro',3)
oro.changeLabel(0,'(0,0)')
oro.changeLabel(1,'(1,1)')
oro.changeLabel(2,'(2,0)')
model.addChanceNode(oro)

# Creamos las aristas de la red bayesiana
model.addArc('Pozo(0,0)', 'Brisa(1,0)')
model.addArc('Pozo(1,1)', 'Brisa(1,0)')
model.addArc('Pozo(2,0)', 'Brisa(1,0)')
model.addArc('Oro', 'Brillo(0,0)')
model.addArc('Oro', 'Brillo(1,1)')
model.addArc('Oro', 'Brillo(2,0)')

# Creamos las probabilidades
model.cpt('Pozo(0,0)')[:]=[0.8,0.2]
model.cpt('Pozo(1,1)')[:]=[0.8,0.2]
model.cpt('Pozo(2,0)')[:]=[0.8,0.2]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 1, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 1, 'Pozo(2,0)': 0}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 0, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 1, 'Pozo(1,1)': 0, 'Pozo(2,0)': 0}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 1, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 1, 'Pozo(2,0)': 0}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 0, 'Pozo(2,0)': 1}] = [0, 1]
model.cpt('Brisa(1,0)')[{'Pozo(0,0)': 0, 'Pozo(1,1)': 0, 'Pozo(2,0)': 0}] = [1, 0]
model.cpt('Oro').fillWith([1/3,1/3,1/3])
model.cpt('Brillo(0,0)')[{'Oro': 0}] = [0,1]
model.cpt('Brillo(0,0)')[{'Oro': 1}] = [1,0]
model.cpt('Brillo(0,0)')[{'Oro': 2}] = [1,0]
model.cpt('Brillo(1,1)')[{'Oro': 0}] = [1,0]
model.cpt('Brillo(1,1)')[{'Oro': 1}] = [0,1]
model.cpt('Brillo(1,1)')[{'Oro': 2}] = [1,0]
model.cpt('Brillo(2,0)')[{'Oro': 0}] = [1,0]
model.cpt('Brillo(2,0)')[{'Oro': 1}] = [1,0]
model.cpt('Brillo(2,0)')[{'Oro': 2}] = [0,1]

# Creamos el nodo de decisión
casilla = gum.LabelizedVariable('Casilla','Casilla a moverse',3)
casilla.changeLabel(0,'(0,0)')
casilla.changeLabel(1,'(1,1)')
casilla.changeLabel(2,'(2,0)')
model.addDecisionNode(casilla)

# Creamos el nodo de utilidad
ut_casilla = gum.LabelizedVariable('UtilityOfCasilla','Valor casilla',1)
model.addUtilityNode(ut_casilla)

# Creamos las aristas de utilidad
model.addArc('Pozo(0,0)', 'UtilityOfCasilla')
model.addArc('Pozo(1,1)', 'UtilityOfCasilla')
model.addArc('Pozo(2,0)', 'UtilityOfCasilla')
model.addArc('Oro', 'UtilityOfCasilla')
model.addArc('Casilla', 'UtilityOfCasilla')

# Asignamos las utilidades
variables = ['Pozo(0,0)','Pozo(1,1)','Pozo(2,0)','Oro','Casilla']
opciones = product([0,1], [0,1], [0,1], ['(0,0)','(1,1)','(2,0)'], ['(0,0)','(1,1)','(2,0)'])
dicts_variables = [{variables[i]:op[i] for i in range(len(variables))} for op in opciones]

def utilidad(d):
    C = d['Casilla']
    if (d['Pozo'+C] == 0) and (d['Oro'] == C):
        return 1
    elif (d['Pozo'+C] == 0):
        return 0
    else:
        return -1

for d in dicts_variables:
    model.utility('UtilityOfCasilla')[d]=utilidad(d)

print("Situación I: Análisis del escenario sin evidencia:")
gnb.showInference(model,evs={},size="6")

Observe que cuando el agente está en la casilla $(1,0)$ y no tiene ninguna información sobre el mundo, todas las tres acciones posibles tienen la misma utilidad. Esto cambia cuando incluimos evidencia. Realice los siguientes ejercicios para analizar distintos esceneraios.

<a class="anchor" id="ej8"></a>**Ejercicio 8:** 

([Anterior ejercicio](#ej7)) ([Próximo ejercicio](#ej9))

Considere un escenario en el cual no hay brillo en $(0,0)$ ni brisa en $(1,0)$. Este es el escenario de la figura de la derecha al comienzo del notebook. En este escenario, ¿cuál acción tiene mayor utilidad esperada? ¿Es esto intuitivamente cierto?

---

<a class="anchor" id="ej9"></a>**Ejercicio 9:** 

([Anterior ejercicio](#ej8)) ([Próximo ejercicio](#ej10))

Considere un escenario en el cual no hay brillo ni pozo en $(0,0)$, pero hay brisa en $(1,0)$. En este escenario, ¿cuál acción tiene mayor utilidad esperada? ¿Es esto intuitivamente cierto?

---

# Agente basado en utilidad  <a class="anchor" id="utility-based">

([Volver al inicio](#inicio))

Ya tenemos todas las herramientas para crear un programa de agente basado en utilidad. Mantendremos igual el método `make_decision()` respecto al agente anterior:
    
<img src="./imagenes/make_decision.png" width="auto"/>

La modificación importante está en el programa de agente:
    
<img src="./imagenes/program_UE.png" width="350"/>
 
Observe que la creación de la red está centrada en la ubicación actual del agente. En efecto, resulta muy ineficiente crear el escenario con todas las variables para brisa, pozo, brillo y oro de todas las rejillas. En efecto, para implementar la función de utilidad requeriríamos una cantidad astronómica de diccionarios (el número exacto es $2^{16}*2^{16}*16*16\approx 7\times 10^{16}$). La solución, la cual resulta bastante buena, como veremos a continuación, es crear una red de decisión para cada casilla. Aún así, necesitamos crear una variable para todos los brillos, pero no para todas las brisas y pozos. Para estas últimas sólo necesitamos considerar casillas adyacentes de manera limitada.
    
El resultado de incluir en la red el oro, los pozos y la briza da lugar al agente basado en utilidad, que presentamos a continuación:

In [ ]:
# Create environment
W = Wumpus(wumpus=(0,3), oro=(3,3), pozos=[(0,2), (1,2), (2,3)])
# Create agent
agente = HeroeUE()
# Create episode
episodio = utils.Episode(environment=W,\
        agent=agente,\
        model_name='Baseline',\
        num_rounds=100)
# Visualize
episodio.renderize()
# Presentar resumen
W.pintar_todo()
W.mensaje

Este agente tiene una dificultad: no reconoce el hedor ni su relación con el Wumpus:

In [ ]:
# Create environment
W = Wumpus(wumpus=(1,1), oro=(2,3), pozos=[(2,1), (3,2), (0,2)])
# Create agent
agente = HeroeUE()
# Create episode
episodio = utils.Episode(environment=W,\
        agent=agente,\
        model_name='Baseline',\
        num_rounds=100)
# Visualize
episodio.renderize()
#episodio.run(verbose=4)
# Presenta resumen
W.pintar_todo()
W.mensaje

No obstante, como baseline, encontramos su medidas de desempeño:

In [ ]:
# Create environment
W = Wumpus(aleatorio=True)
# Create agent
agente = HeroeUE()
# Create episode
episodio = utils.Episode(environment=W,\
        agent=agente,\
        model_name='Baseline',\
        num_rounds=100)
# Run simulation and save the data
df = episodio.simulate(num_episodes=100)
# Plot histogram
p = utils.Plot(df)
p.plot_histogram_rewards(file='baseline.png')

<a class="anchor" id="ej10"></a>**Ejercicio 10:** 

([Anterior ejercicio](#ej9))

En la clase `HeroeUE` modifique el método `crear_red_decision()` para incluir las variables de hedor y Wumpus, de tal manera que el héroe sea capaz de superar el escenario anterior. Evalúe el desempeño del agente sobre 100 escenarios aleatorios (fije en 100 el número máximo de pasos). Como mínimo, el agente debe explorar de manera segura la cueva sin morir en el intento.

En este notebook usted aprendió:

* Implementar redes bayesianas mediante el paquete `pyAgrum`.
* Realizar inferencias sobre probabilidades con o sin evidencia adicional mediante redes bayesianas.
* Asociar las acciones racionales con la maximización de la utilidad esperada.
* Implementar redes de decisión mediante el paquete `pyAgrum`.
* Implementar un programa de agente basado en la utilidad.